<a href="https://colab.research.google.com/github/zamanmiraz/DSandML-Notebooks/blob/main/RAG/01_simple_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/guyernest/advanced-rag.git
%cd advanced-rag
!pip install --upgrade -r requirements.txt
# Install a compatible version of torchvision to address the nms error
!pip install torchvision==0.18.0
!pip install -q -U google-generativeai

Cloning into 'advanced-rag'...
remote: Enumerating objects: 281, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 281 (delta 4), reused 10 (delta 3), pack-reused 268 (from 1)
Receiving objects: 100% (281/281), 18.84 MiB | 12.75 MiB/s, done.
Resolving deltas: 100% (167/167), done.
/content/advanced-rag
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.7/163.7 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [41]:
from rich.console import Console
from rich.style import Style
import pathlib
from rich_theme_manager import Theme, ThemeManager

THEMES = [
    Theme(
        name="dark",
        description="Dark mode theme",
        tags=["dark"],
        styles={
            "repr.own": Style(color="#e87d3e", bold=True),      # Class names
            "repr.tag_name": "dim cyan",                        # Adjust tag names
            "repr.call": "bright_yellow",                       # Function calls and other symbols
            "repr.str": "bright_green",                         # String representation
            "repr.number": "bright_red",                        # Numbers
            "repr.none": "dim white",                           # None
            "repr.attrib_name": Style(color="#e87d3e", bold=True),    # Attribute names
            "repr.attrib_value": "bright_blue",                 # Attribute values
            "default": "bright_white on black"                  # Default text and background
        },
    ),
    Theme(
        name="light",
        description="Light mode theme",
        styles={
            "repr.own": Style(color="#22863a", bold=True),          # Class names
            "repr.tag_name": Style(color="#00bfff", bold=True),     # Adjust tag names
            "repr.call": Style(color="#ffff00", bold=True),         # Function calls and other symbols
            "repr.str": Style(color="#008080", bold=True),          # String representation
            "repr.number": Style(color="#ff6347", bold=True),       # Numbers
            "repr.none": Style(color="#808080", bold=True),         # None
            "repr.attrib_name": Style(color="#ffff00", bold=True),  # Attribute names
            "repr.attrib_value": Style(color="#008080", bold=True), # Attribute values
            "default": Style(color="#000000", bgcolor="#ffffff"),   # Default text and background
        },
    ),
]

theme_dir = pathlib.Path("themes").expanduser()
theme_dir.expanduser().mkdir(parents=True, exist_ok=True)

theme_manager = ThemeManager(theme_dir=theme_dir, themes=THEMES)
theme_manager.list_themes()

dark = theme_manager.get("dark")
theme_manager.preview_theme(dark)

 Theme  Description       Tags  Path               
 dark   Dark mode theme   dark  themes/dark.theme  
 light  Light mode theme        themes/light.theme

                                      Theme: dark - themes/dark.theme                                      
┌───────────────────┬───────────────┬───────┬─────────┬─────────┬────────────────┬────────────────────────┐
│ style             │ color         │ color │ bgcolor │ bgcolor │ attributes     │ example                │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ default           │ bright_white  │ █████ │ black   │ █████   │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.attrib_name  │ #e87d3e       │ █████ │ None    │         │ b------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.attrib_value │ bright_blue   │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.call         │ bright_yellow │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.none         │ white         │ █████ │ None    │         │ -d------------ │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.number       │ bright_red    │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.own          │ #e87d3e       │ █████ │ None    │         │ b------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.str          │ bright_green  │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.tag_name     │ cyan          │ █████ │ None    │         │ -d------------ │ The quick brown fox... │
└───────────────────┴───────────────┴───────┴─────────┴─────────┴────────────────┴────────────────────────┘
┌─ attributes legend ──────────────────────────────────────────────────────────────────┐
│  b: bold, d: dim, i: italic, u: underline, U: double underline, B: blink, 2: blink2  │
│  r: reverse, c: conceal, s: strike, f: frame, e: encircle, o: overline, L: Link      │
└──────────────────────────────────────────────────────────────────────────────────────┘

In [42]:
from rich.console import Console

dark = theme_manager.get("dark")
# Create a console with the dark theme
console = Console(theme=dark)

In [3]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
data = (
    pd
    .read_csv('advanced-rag/data/top_rated_wines.csv')
    .query('variety.notna()')
    .reset_index(drop=True)
    .to_dict(orient='records')
)
data[:2]

[{'name': '3 Rings Reserve Shiraz 2004',
  'region': 'Barossa Valley, Barossa, South Australia, Australia',
  'variety': 'Red Wine',
  'rating': 96.0,
  'notes': 'Vintage Comments : Classic Barossa vintage conditions. An average wet Spring followed by extreme heat in early February. Occasional rainfall events kept the vines in good balance up to harvest in late March 2004. Very good quality coupled with good average yields. More than 30 months in wood followed by six months tank maturation of the blend prior to bottling, July 2007. '},
 {'name': 'Abreu Vineyards Cappella 2007',
  'region': 'Napa Valley, California',
  'variety': 'Red Wine',
  'rating': 96.0,
  'notes': 'Cappella is a proprietary blend of two clones of Cabernet Sauvignon with Cabernet Franc, Petit Verdot and Merlot. The gravelly soil at Cappella produces fruit that is very elegant in structure. The resulting wine exhibits beautiful purity of fruit with fine grained and lengthy tannins. '}]

In [1]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

# create the vector database client
qdrant = QdrantClient(":memory:")

# create the embedding encoder
encoder = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# Create the collection to store the wine rating data
collection_name = "top_wines"
qdrant.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE
    )
)

True

In [9]:
# vectorize!
qdrant.upload_points(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data)
    ]
)

In [10]:
qdrant.get_collection(collection_name=collection_name)

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=1347, segments_count=1, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=None, sharding_method=None, replication_factor=None, write_consistency_factor=None, read_fan_out_factor=None, on_disk_payload=None, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0

In [11]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"
query_vector = encoder.encode(user_prompt).tolist()

In [12]:
# search time for awesome wines

hits = qdrant.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3
)

In [14]:
from rich.console import Console
from rich.text import Text
from rich.table import Table
table = Table(title="Retrieval Results", show_lines=True)

table.add_column("Name", style="#e0e0e0")
table.add_column("Region", style="bright_red")
table.add_column("Variety", style="green")
table.add_column("Rating", style="yellow")
table.add_column("Notes", style="#89ddff")
table.add_column("Score", style="#a6accd")

for hit in hits:
    table.add_row(
        hit.payload["name"],
        hit.payload["region"],
        hit.payload["variety"],
        str(hit.payload["rating"]),
        f'{hit.payload["notes"][:50]}...',
        f"{hit.score:.4f}"
    )

table

                                                 Retrieval Results                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Name                           ┃ Region           ┃ Variety  ┃ Rating ┃ Notes                          ┃ Score  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Catena Zapata Argentino        │ Argentina        │ Red Wine │ 98.0   │ "The single-vineyard 2004      │ 0.6378 │
│ Vineyard Malbec 2004           │                  │          │        │ Malbec Argentino Vineyar...    │        │
├────────────────────────────────┼──────────────────┼──────────┼────────┼────────────────────────────────┼────────┤
│ Bodega Colome Altura Maxima    │ Salta, Argentina │ Red Wine │ 96.0   │ Winemaker Thibaut Delmotte has │ 0.6180 │
│ Malbec 2012                    │                  │          │        │ crafted wines of di...         │        │
├────────────────────────────────┼──────────────────┼──────────┼────────┼────────────────────────────────┼────────┤
│ Catena Zapata Adrianna         │ Argentina        │ Red Wine │ 97.0   │ "The single-vineyard 2004      │ 0.6118 │
│ Vineyard Malbec 2004           │                  │          │        │ Malbec Adrianna Vineyard...    │        │
└────────────────────────────────┴──────────────────┴──────────┴────────┴────────────────────────────────┴────────┘

In [15]:
from dotenv import load_dotenv
load_dotenv()

False

In [16]:
from openai import OpenAI
from rich.panel import Panel

client = OpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

# Task
Set up Gemini API using Colab Secrets Manager.

First, install the Google Generative AI library.

In [19]:
!pip install -q -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 71.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.62.3 requires protobuf<5.0dev,>=4.21.6, but you have protobuf 5.29.5 which is incompatible.
dataproc-spark-connect 0.8.3 requires tqdm>=4.67, but you have tqdm 4.66.5 which is incompatible.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.27.2 which is incompatible.
google-adk 1.15.1 requires anyio<5.0.0,>=4.9.0; python_version >= "3.10", but you have anyio 4.6.0 which is incompatible.
google-adk 1.15.1 requires requests<3.0.0,>=2.32.4, but you have requests 2.32.3 which is incompatible.


In [31]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Next, initialize the Gemini model. You can choose a specific model depending on your needs.

In [37]:
# Initialize the Gemini API
gemini_model = genai.GenerativeModel('gemini-2.0-flash')

Finally, you can test the API to make sure it's working correctly.

In [43]:
# Now time to connect to the large language model
import google.generativeai as genai
from rich.panel import Panel
from rich.text import Text

# Assuming gemini_model is already initialized in a previous cell
# gemini_model = genai.GenerativeModel('gemini-1.5-flash')

response = gemini_model.generate_content(f"As a wine specialist, suggest an amazing Malbec wine from Argentina based on the following user prompt: {user_prompt}")

response_text = Text(response.text)
styled_panel = Panel(
    response_text,
    title="Wine Recommendation with Gemini",
    expand=False,
    border_style="bold green",
    padding=(1, 1)
)

console.print(styled_panel)

╭──────────────────────────────────────── Wine Recommendation with Gemini ────────────────────────────────────────╮
│                                                                                                                 │
│ Okay, you want an amazing Malbec from Argentina? Fantastic choice! Based on your request for just *an* amazing  │
│ Malbec (meaning you're open to possibilities), I'm going to suggest the **Catena Zapata Adrianna Vineyard       │
│ Mundus Bacillus Terrae Malbec**.                                                                                │
│                                                                                                                 │
│ Here's why I think it's a fantastic recommendation and why it elevates above "just good" to "amazing":          │
│                                                                                                                 │
│ *   **Unparalleled Vineyard Source:** The Adrianna Vineyard is legendary. Located in Gualtallary, Uco Valley at │
│ a staggering altitude of almost 5,000 feet (1,500 meters) above sea level, it's one of the highest and most     │
│ extreme vineyards in Mendoza. This extreme altitude results in:                                                 │
│     *   **Intense Sunlight:** Thick skins on the grapes, leading to concentrated flavors and tannins.           │
│     *   **Cool Nights:** Preserve acidity and freshness, crucial for balance and aging potential.               │
│     *   **Unique Soil Profile:** The "Mundus Bacillus Terrae" parcel, in particular, boasts a unique soil       │
│ composition, further adding to the wine's distinctive characteristics.                                          │
│                                                                                                                 │
│ *   **Winemaking Philosophy:** The Catena Zapata family is dedicated to understanding the terroir of Mendoza.   │
│ They approach winemaking with a blend of tradition and innovation, focusing on minimal intervention to allow    │
│ the fruit to express itself fully. They use indigenous yeasts, age the wine in neutral oak and concrete, all in │
│ the pursuit of the best possible expression of the Adrianna vineyard.                                           │
│                                                                                                                 │
│ *   **Complex Flavor Profile:** Expect a multifaceted flavor profile that goes beyond the typical dark fruit    │
│ found in many Malbecs. This wine will present:                                                                  │
│     *   **Aromas:** Violet, lavender, black raspberry, black cherry, with a subtle earthy minerality.           │
│     *   **Palate:** A layered texture, with a core of dark fruit intertwined with savory notes, spice, and      │
│ elegant tannins. A long and persistent finish.                                                                  │
│                                                                                                                 │
│ *   **Age-Worthiness:** While enjoyable in its youth, this Malbec has the structure and complexity to age       │
│ gracefully for 10-20+ years.                                                                                    │
│                                                                                                                 │
│ *   **Recognition:** This wine consistently receives high scores and accolades from critics worldwide, further  │
│ solidifying its reputation.                                                                                     │
│                                                                                                                 │
│ *   **Story and Legacy:** The Catena Zapata family has played a crucial role in the rise of Argentine Malbec.   │
│ This wine is a testament to their dedication and pione

In [44]:

# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [45]:
# Use Gemini for Retrieval Augmented Generation (RAG)

completion = gemini_model.generate_content(
    f"As a wine specialist, suggest an amazing Malbec wine from Argentina based on the following user prompt and the provided search results:\n\nUser Prompt: {user_prompt}\n\nSearch Results: {search_results}"
)

response_text = Text(completion.text)
styled_panel = Panel(
    response_text,
    title="Wine Recommendation with Retrieval Augmented Generation (Gemini)",
    expand=False,
    border_style="bold blue",
    padding=(1, 1)
)

console.print(styled_panel)

╭─────────────────────── Wine Recommendation with Retrieval Augmented Generation (Gemini) ────────────────────────╮
│                                                                                                                 │
│ Based on your prompt and the search results, I highly recommend the **Catena Zapata Argentino Vineyard Malbec   │
│ 2004**.                                                                                                         │
│                                                                                                                 │
│ Here's why:                                                                                                     │
│                                                                                                                 │
│ *   **Highest Rating:** It boasts the highest rating in the results at 98 points, indicating exceptional        │
│ quality.                                                                                                        │
│ *   **Exceptional Achievement:** The notes explicitly call it "an exceptional achievement in Malbec."           │
│ *   **Aging Potential:** It's built for the long haul, described as needing a "minimum of a decade of           │
│ additional cellaring" and expected to age gracefully for 25-40 years. This signifies its superb structure and   │
│ complexity.                                                                                                     │
│ *   **Winemaker/Winery Reputation:** Catena Zapata is referred to as "the Argentina winery of reference – the   │
│ standard of excellence for comparing all others." This speaks volumes about the winery's consistent dedication  │
│ to producing top-tier wines.                                                                                    │
│ *   **Complex Aromatics:** The detailed description of its aromas - "wood smoke, creosote, pepper, clove, black │
│ cherry, and blackberry" - suggests a deeply complex and layered wine experience.                                │
│                                                                                                                 │
│ While the other options are also excellent, the Argentino Vineyard Malbec 2004 stands out due to its superior   │
│ rating, exceptional aging potential, and the unparalleled reputation of Catena Zapata. If you're looking for a  │
│ truly amazing and unforgettable Malbec experience from Argentina, this is a fantastic choice. Be prepared to    │
│ cellar it, or decant it well in advance if you choose to drink it soon to allow it to fully express itself.     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯